# LLM Parameter tuning with Optuna

References:
* https://huggingface.co/datasets/ai4privacy/pii-masking-300k
* https://huggingface.co/neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a16

In [ ]:
%pip install -q huggingface-hub==0.23.2
%pip install -q transformers==4.40.2
#%pip install -q accelerate==0.30.1
#%pip install -q sentencepiece==0.2.0
%pip install -q datasets==2.19.1
#%pip install -q sentence-transformers==2.7.0
%pip install -q jiwer==3.0.4
%pip install optuna==3.6.1
%pip install optimum[graphcore]==1.21.2
%pip install auto-gptq==0.7.1

In [1]:
import os
import yaml
import numpy as np
import optuna
import requests

from jiwer import wer
from tqdm import tqdm
from datasets import load_dataset

In [2]:
from google.colab import drive
from getpass import getpass

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read YAML file
f_path = "/content/drive/MyDrive/GitHub/python-codebase/machine_learning/private_keys.yml"
with open(f_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)
os.environ['HF_API_TOKEN'] = data_loaded['HF_API_KEY']
os.environ['GITHUB_TOKEN'] = data_loaded['GITHUB_TOKEN']

In [4]:
# Set up token
from huggingface_hub import login
login(token=os.environ['HF_API_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from transformers.utils import is_auto_gptq_available, is_optimum_available
print(is_auto_gptq_available())
print(is_optimum_available())

True
True


## 0. General functions

In [ ]:
'''
# Define a function to perform text generation and evaluate WER
def evaluate_model(model, tokenizer, input_list, output_list, device, dct_params):
    model.eval()
    predictions = []

    for example in input_list:
      input_ids = tokenizer.encode(example["text"], return_tensors="pt").to(device)
      with torch.no_grad():
         output_ids = model.generate(input_ids, **dct_params)
      generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
      predictions.append(generated_text)

    return wer(predictions, output_list)
'''

In [ ]:
'''
## Inference with the pre trained bloom model
# this function returns the outputs from the model received, and inputs.
def get_outputs(
    model,
    inputs,
    dct_params = {
        'max_new_tokens':100,
        #'early_stopping':True
        }
    ):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        #max_new_tokens=max_new_tokens,
        # temperature=0.2,
        # top_p=0.95,
        # do_sample=True,
        #repetition_penalty=1.5,  # Avoid repetition.
        eos_token_id=tokenizer.eos_token_id,
        **dct_params
    )
    return outputs
'''

In [6]:
# Example (directly accesing HF endpoint w/ requests)
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def get_output(input_prompt, dct_params={'max_new_tokens': 250, 'temperature': 0.1}):
  data = query(
      {
          'inputs': input_prompt,
          'parameters': dct_params
      }
  )
  return data[0]['generated_text']

In [18]:
# Define a function to perform text generation and evaluate WER
def evaluate_model(input_list, output_list, dct_params):
  predictions = []
  for example in tqdm(input_list):
    iter_pred = get_output(example, dct_params=dct_params)
    predictions.append(iter_pred)
  return wer(predictions, output_list)

## 1. Load dataset

In [8]:
dataset = load_dataset("ai4privacy/pii-masking-300k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/177677 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47728 [00:00<?, ? examples/s]

In [9]:
# Check data
print(dataset.keys())
print(list(set(dataset['train']['language'])))
print(dataset['train'])
print("*"*75)
print(dataset['train']['source_text'][0])
print("*"*75)
print(dataset['train']['target_text'][0])
print("*"*75)

dict_keys(['train', 'validation'])
['English', 'French', 'German', 'Spanish', 'Dutch', 'Italian']
Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 177677
})
***************************************************************************
Subject: Group Messaging for Admissions Process

Good morning, everyone,

I hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:

- wynqvrh053 - Meeting at 10:20am
- luka.burg - Meeting at 21
- qahil.wittauer - Meeting at quarter past 13
- gholamhossein.ruschke - Meeting at 9:47 PM
- pdmjrsyoz1460 
***************************************************************************
Subject: Group Messaging for Admissions Process

Good morning, everyone,

I hope this message finds you well. As we co

In [10]:
# Create datasets
N_SAMPLES_TRAIN = 1000
N_SAMPLES_VAL   = 100
LANG = 'English' # 'Spanish'

count_train = 0
count_val = 0

# Initialize lists to store results
input_train  = []
target_train = []
input_val    = []
target_val   = []

# Iterate through the training dataset
for source, target, lang in tqdm(
    zip(
        dataset['train']['source_text'],
        dataset['train']['target_text'],
        dataset['train']['language']
        ),
    total = len(dataset['train']['source_text'])
    ):
    if lang == LANG:
        input_train.append(source)
        target_train.append(target)
        count_train += 1
        if count_train >= N_SAMPLES_TRAIN:
            break

# Iterate through the validation dataset
for source, target, lang in tqdm(
    zip(
        dataset['validation']['source_text'],
        dataset['validation']['target_text'],
        dataset['validation']['language']
        ),
    total = len(dataset['validation']['source_text'])
    ):
    if lang == LANG:
        input_val.append(source)
        target_val.append(target)
        count_val += 1
        if count_val >= N_SAMPLES_VAL:
            break

  0%|          | 99/47728 [00:00<00:00, 375439.51it/s]


In [11]:
print("*"*75)
print(input_train[3])
print("*"*75)
print(target_train[3])
print("*"*75)

***************************************************************************
Card: KB90324ER\n   Country: GB\n   Building: 163\n   Street: Conygre Grove\n   City: Bristol\n   State: ENG\n   Postcode: BS34 7HU, BS34 7HZ\n   Password: q4R\\\n\n2. Applicant: Baasgaran Palmoso\n   Email: blerenbaasgara@gmail.com\n   Social Number: 107-393-9036\n   ID Card: SC78428CU\n   Country: United Kingdom\n   Building: 646\n   Street: School Lane\n   City: Altrincham\n   State: ENG\n   Postcode: WA14 5R
***************************************************************************
Card: [IDCARD]\n   Country: [COUNTRY]\n   Building: [BUILDING]\n   Street: [STREET]\n   City: [CITY]\n   State: [STATE]\n   Postcode: [POSTCODE]\n   Password: [PASS]\n\n2. Applicant: [LASTNAME1] [LASTNAME2]\n   Email: [EMAIL]\n   Social Number: [SOCIALNUMBER]\n   ID Card: [IDCARD]\n   Country: [COUNTRY]\n   Building: [BUILDING]\n   Street: [STREET]\n   City: [CITY]\n   State: [STATE]\n   Postcode: WA14 5R
***********************

## 2. LLM model

In [ ]:
'''
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
pipe(messages)
'''

In [12]:
# https://huggingface.co/docs/api-inference/en/detailed_parameters
data = query(
    {
        'inputs': "Can you please let us know more details about your ",
        'parameters': {
            'max_new_tokens': 250,
            'temperature': 0.1
        }
    }
)
print(data)

[{'generated_text': 'Can you please let us know more details about your 2019-2020 budget?\n\nOur 2019-2020 budget is focused on delivering high-quality, affordable education to our students while maintaining the financial stability of the university. Here are some key points:\n\n1. Tuition and Fees: We have kept tuition and fees as low as possible while still providing a top-notch education. For undergraduate students, the tuition and fees for the 2019-2020 academic year is $10,500 per semester for in-state students and $26,000 per semester for out-of-state students.\n\n2. Financial Aid: We are committed to making our university accessible to students from all backgrounds. In the 2019-2020 academic year, we will provide over $100 million in financial aid to our students, including grants, scholarships, and work-study programs.\n\n3. Faculty and Staff: We are investing in our faculty and staff to ensure that they have the resources they need to provide the best possible education to our

In [ ]:
'''
# Use a pipeline as a high-level helper
from transformers import pipeline

model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w4a16"
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model=model_name)
pipe(messages)
'''

In [ ]:
'''
# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
# model_name = "bigscience/bloom-1b1"
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w4a16"
NUM_VIRTUAL_TOKENS = 4
NUM_EPOCHS = 6

tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
)
'''

In [25]:
# Generate another example for this sentence
prompt_template = """
  You are a privacy expert. Identify the sensitive information within a text and mask it.

  **Example:**

  Input:
  ```text
  Good morning Pablo. I'll see you in Madrid at 11:00 a.m.
  ```

  Output:
  ```text
  Good morning [USERNAME]. I'll see you in [CITY] at [TIME]
  ```

  **Considerations**
  * Only return the text part of the output, nothing else.
  * Do not include any additional explanations.

  **Input data**
  Input:
"""

input_prompt = f"""
  {prompt_template}
  ```text
  {input_train[3]}
  ```

  Output:
  ```text
"""
print(input_prompt)


  
  You are a privacy expert. Identify the sensitive information within a text and mask it.

  **Example:**

  Input:
  ```text
  Good morning Pablo. I'll see you in Madrid at 11:00 a.m.
  ```

  Output:
  ```text
  Good morning [USERNAME]. I'll see you in [CITY] at [TIME]
  ```

  **Considerations**
  * Only return the text part of the output, nothing else.
  * Do not include any additional explanations.

  **Input data**
  Input:

  ```text
  Card: KB90324ER\n   Country: GB\n   Building: 163\n   Street: Conygre Grove\n   City: Bristol\n   State: ENG\n   Postcode: BS34 7HU, BS34 7HZ\n   Password: q4R\\\n\n2. Applicant: Baasgaran Palmoso\n   Email: blerenbaasgara@gmail.com\n   Social Number: 107-393-9036\n   ID Card: SC78428CU\n   Country: United Kingdom\n   Building: 646\n   Street: School Lane\n   City: Altrincham\n   State: ENG\n   Postcode: WA14 5R
  ```

  Output:
  ```text



In [29]:
data = query(
    {
        'inputs': input_prompt,
        'parameters': {
            'max_new_tokens': 250,
            'temperature': 0.1
        }
    }
)
print(data[0]['generated_text'])


  
  You are a privacy expert. Identify the sensitive information within a text and mask it.

  **Example:**

  Input:
  ```text
  Good morning Pablo. I'll see you in Madrid at 11:00 a.m.
  ```

  Output:
  ```text
  Good morning [USERNAME]. I'll see you in [CITY] at [TIME]
  ```

  **Considerations**
  * Only return the text part of the output, nothing else.
  * Do not include any additional explanations.

  **Input data**
  Input:

  ```text
  Card: KB90324ER\n   Country: GB\n   Building: 163\n   Street: Conygre Grove\n   City: Bristol\n   State: ENG\n   Postcode: BS34 7HU, BS34 7HZ\n   Password: q4R\\\n\n2. Applicant: Baasgaran Palmoso\n   Email: blerenbaasgara@gmail.com\n   Social Number: 107-393-9036\n   ID Card: SC78428CU\n   Country: United Kingdom\n   Building: 646\n   Street: School Lane\n   City: Altrincham\n   State: ENG\n   Postcode: WA14 5R
  ```

  Output:
  ```text
  Card: [CARD_NUMBER]\n   Country: [COUNTRY]\n   Building: [BUILDING]\n   Street: [STREET]\n   City: [CITY

## 3. Hyperparameter tuning

In [32]:
# Define the objective function for Optuna
def objective(trial, input_list, output_list):
    # Hyperparameters to tune
    max_new_tokens = trial.suggest_int('max_new_tokens', 250, 500)
    temperature = trial.suggest_loguniform('temperature', 0.1, 0.4)
    dct_params = {'max_new_tokens': max_new_tokens, 'temperature': temperature}

    # Evaluate the model
    validation_wer = evaluate_model(input_list, output_list, dct_params)

    return validation_wer

In [3]:
'''
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5, show_progress_bar=True)
'''

In [33]:
# Create a function to optimize hyperparameters with a fixed parameter
def optimize_hyperparameters_with_fixed_param(input_list, output_list):
    # Create an Optuna study
    study = optuna.create_study(direction="minimize")

    # Optimize the objective function with the fixed parameter
    study.optimize(lambda trial: objective(trial, input_list, output_list), n_trials=5, show_progress_bar=True)

    # Print the best hyperparameters
    print("Best hyperparameters: ", study.best_params)

    return study.best_params

# Run the optimization with a fixed parameter
dct_best_params = optimize_hyperparameters_with_fixed_param(input_train[:5], target_train[:5])

[I 2024-07-18 19:29:13,086] A new study created in memory with name: no-name-bce5062a-b783-489f-8b23-fffeba62f820


  0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-32-76929c3e4e66>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  temperature = trial.suggest_loguniform('temperature', 0.1, 0.4)

100%|██████████| 5/5 [00:15<00:00,  3.04s/it]


[I 2024-07-18 19:29:28,340] Trial 0 finished with value: 0.5905797101449275 and parameters: {'max_new_tokens': 402, 'temperature': 0.21003457665952502}. Best is trial 0 with value: 0.5905797101449275.



100%|██████████| 5/5 [00:14<00:00,  2.84s/it]


...

[I 2024-07-18 19:29:57,905] Trial 2 finished with value: 0.6891334250343879 and parameters: {'max_new_tokens': 481, 'temperature': 0.11034326259987146}. Best is trial 0 with value: 0.5905797101449275.



100%|██████████| 5/5 [00:18<00:00,  3.63s/it]


[I 2024-07-18 19:30:16,063] Trial 3 finished with value: 0.6691068814055637 and parameters: {'max_new_tokens': 386, 'temperature': 0.2628199216107374}. Best is trial 0 with value: 0.5905797101449275.



100%|██████████| 5/5 [00:10<00:00,  2.15s/it]


[I 2024-07-18 19:30:26,837] Trial 4 finished with value: 0.6412698412698413 and parameters: {'max_new_tokens': 276, 'temperature': 0.1312452807474583}. Best is trial 0 with value: 0.5905797101449275.
Best hyperparameters:  {'max_new_tokens': 402, 'temperature': 0.21003457665952502}


In [34]:
data = query(
    {
        'inputs': input_prompt,
        'parameters': dct_best_params
    }
)
print(data[0]['generated_text'])


  
  You are a privacy expert. Identify the sensitive information within a text and mask it.

  **Example:**

  Input:
  ```text
  Good morning Pablo. I'll see you in Madrid at 11:00 a.m.
  ```

  Output:
  ```text
  Good morning [USERNAME]. I'll see you in [CITY] at [TIME]
  ```

  **Considerations**
  * Only return the text part of the output, nothing else.
  * Do not include any additional explanations.

  **Input data**
  Input:

  ```text
  Card: KB90324ER\n   Country: GB\n   Building: 163\n   Street: Conygre Grove\n   City: Bristol\n   State: ENG\n   Postcode: BS34 7HU, BS34 7HZ\n   Password: q4R\\\n\n2. Applicant: Baasgaran Palmoso\n   Email: blerenbaasgara@gmail.com\n   Social Number: 107-393-9036\n   ID Card: SC78428CU\n   Country: United Kingdom\n   Building: 646\n   Street: School Lane\n   City: Altrincham\n   State: ENG\n   Postcode: WA14 5R
  ```

  Output:
  ```text
  Card: [CARD_NUMBER]\n   Country: [COUNTRY]\n   Building: [BUILDING]\n   Street: [STREET]\n   City: [CITY

In [ ]:
# Run the optimization with a fixed parameter
optimize_hyperparameters_with_fixed_param(input_train[:10], target_train[:10])